In [ ]:
src_url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
src_zip = "dakshina_dataset_v1.0.tar"
DATA_SRC="dakshina_dataset_v1.0/ta/lexicons"
DATA_TRAIN_SRC = "/ta.translit.sampled.train.tsv"
DATA_VAL_SRC = "/ta.translit.sampled.dev.tsv" 
DATA_TEST_SRC = "/ta.translit.sampled.test.tsv"
#TRAIN_IMAGES_PER_LABEL = 1000
#TEST_IMAGES_PER_LABEL = 200
BALANCED_SPLITS = {"train" : 900, "val" : 100}
PROJECT_NAME = "CS6910 ASSIGNMENT 3"
dataset='dakshina-dataset'

In [ ]:
%%capture
!curl -SL $src_url > $src_zip
!tar -xf $src_zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import pandas as pd
import keras
from keras.layers import Input, LSTM, Dense, Embedding, GRU, SimpleRNN, Dropout, Activation, dot, concatenate, TimeDistributed
from keras.models import Model

!pip3 install tensorflow -qqq
!pip3 install wandb -qqq
import wandb
!wandb login
from wandb.keras import WandbCallback

wandb: Currently logged in as: cs6910krsrd (use `wandb login --relogin` to force relogin)


In [ ]:
import matplotlib.font_manager as fm

!wget https://github.com/arul20be/Latha-Tamil-Font/blob/master/system/fonts/NotoSansTamil-Regular.ttf
fm.fontManager.ttflist += fm.createFontList(['NotoSansTamil-Regular.ttf'])
plt.rc('font', family='NotoSansTamil-Regular-Regular')
!mv NotoSansTamil-Regular.ttf /usr/share/fonts/truetype/liberation/

--2021-05-19 22:06:25--  https://github.com/arul20be/Latha-Tamil-Font/blob/master/system/fonts/NotoSansTamil-Regular.ttf
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘NotoSansTamil-Regular.ttf.3’

NotoSansTamil-Regul     [ <=>                ] 109.08K  --.-KB/s    in 0.007s  

2021-05-19 22:06:25 (14.6 MB/s) - ‘NotoSansTamil-Regular.ttf.3’ saved [111702]



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  after removing the cwd from sys.path.


In [ ]:
!fc-list :lang=en

/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Italic.ttf: Liberation Sans Narrow:style=Italic
/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf: Liberation Sans:style=Regular
/usr/share/fonts/truetype/liberation/LiberationMono-BoldItalic.ttf: Liberation Mono:style=Bold Italic
/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf: Liberation Serif:style=Italic
/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf: Liberation Mono:style=Bold
/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf: Liberation Sans Narrow:style=Regular
/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf: Liberation Serif:style=Bold
/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf: Liberation Mono:style=Regular
/usr/share/fonts/truetype/liberation/LiberationSans-Italic.ttf: Liberation Sans:style=Italic
/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf: Liberation Serif:style=Bold Italic
/usr/share/fonts/truet

Uploading Data: only run once

In [ ]:

run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(dataset, type="raw_data")

raw_data_at.add_dir(DATA_SRC)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

Downloading Data

In [ ]:
run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="download")

# Query W&B for an artifact and mark it as input to this run
artifact = run.use_artifact(dataset+':latest')

# Download the artifact's contents
artifact_dir = artifact.download()
run.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


In [ ]:
colnames=["ntv","rmn",'nAtt'] #native, romanized and number of attestations
df_train = pd.read_csv(artifact_dir + DATA_TRAIN_SRC,sep="\t",names=colnames,na_filter=False)
df_val = pd.read_csv(artifact_dir + DATA_VAL_SRC,sep="\t",names=colnames,na_filter=False)
MODEL_NAME = "Seq2Seq"
FINAL_MODEL_DIR = "trained_model"


batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding and decoding space (LSTM/GRU/RNN)
num_samples = 10000  # Number of samples to train on.
embed_dim=16  #embedding size
n_encoder=2 #number of encoder layers 
n_decoder=2 #number of decoder layers
cell_type="LSTM"#"GRU" "RNN" # Cell type of encoder and decoder
do=0.2 #dropout
beam_size=3

input_texts = df_train.rmn.to_list()    # input words(romanized)
target_texts = df_train.ntv.apply(lambda s:'\t'+s+'\n').to_list() # target words(native)
input_characters = set(df_train.rmn.sum()) # input vocabulary (all english letters)
target_characters = set(df_train.ntv.sum()) # target vocabulary (all tamil letters)
val_input_texts = df_val.rmn.to_list()    # input words(romanized)
val_target_texts = df_val.ntv.apply(lambda s: s).to_list() # target words(native)
valid_target_texts = df_val.ntv.apply(lambda s:'\t'+s+'\n').to_list() # target words(native)


input_characters = sorted(list(input_characters)) 
input_characters.append(' ')
target_characters = sorted(list(target_characters))
target_characters.append(' ')
target_characters.append('\t')
target_characters.append('\n')



num_encoder_tokens = len(input_characters)   # size of input vocabulary
num_decoder_tokens = len(target_characters)   # ize of target vocabulary
max_encoder_seq_length = max([len(txt) for txt in input_texts])  # max input word size
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # max output word size

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
 
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)]) #dict mapping input letters to integers
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)]) #dict mapping output letters to integers

encoder_input_data = np.zeros(                                              
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)

val_encoder_input_data = np.zeros(                                              
    (len(val_input_texts), max_encoder_seq_length), dtype="float32"
)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length,len(target_token_index)), dtype="float32"
)

val_decoder_input_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length), dtype="float32"
)
val_decoder_target_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length,len(target_token_index)), dtype="float32"
)

CELL={"LSTM":LSTM,"GRU":GRU,"RNN":SimpleRNN}

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

Number of samples: 68218
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for inputs: 30
Max sequence length for outputs: 28


In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] =  target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1,target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :] = target_token_index[" "]
    decoder_target_data[i, t:,target_token_index[" "]] =  1.0

for i, (input_text, target_text) in enumerate(zip(val_input_texts, valid_target_texts)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t] = input_token_index[char]
    val_encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t] =  target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1,target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1 :] = target_token_index[" "]
    val_decoder_target_data[i, t:,target_token_index[" "]] =  1.0

In [ ]:
def train():
  run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="train")
  CELL={"LSTM":LSTM,"GRU":GRU,"RNN":SimpleRNN}

  cfg = wandb.config
  batch_size = cfg.batch_size  # Batch size for training.
  epochs = cfg.epochs  # Number of epochs to train for.
  latent_dim = cfg.latent_dim  # Latent dimensionality of the encoding and decoding space (LSTM/GRU/RNN)
  num_samples = cfg.num_samples  # Number of samples to train on.
  embed_dim=cfg.embed_dim#embedding size
  n_encoder=cfg.n_encoder #number of encoder layers 
  n_decoder=cfg.n_decoder #number of decoder layers
  cell_type=cfg.cell_type#"GRU" "RNN" # Cell type of encoder and decoder
  do=cfg.drop_out #dropout
  beam_size=cfg.beam_size
  # Define an input sequence and process it.
  encoder_inputs=Input(shape=(None,))
  x = Embedding(num_encoder_tokens, embed_dim)(encoder_inputs)
  encoder_states = []
  print(CELL)
  for j in range(n_encoder)[:-1]:
    x = (CELL[cell_type](latent_dim,dropout=do,return_state=True,return_sequences=True)(x)) [0]
  temp = CELL[cell_type](latent_dim,dropout=do,return_state=True,return_sequences=True)(x)
  x=temp[0]
  encoder_states = temp[1:]

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None,))
  out_embed=Embedding(num_decoder_tokens, embed_dim)
  embedded_word=out_embed(decoder_inputs)
  x=embedded_word
  output_layers = []
  for j in range(n_decoder):
    output_layers.append(
        CELL[cell_type](latent_dim,dropout=do, return_sequences=True,return_state=True)
    )
    temp = output_layers[-1](x, initial_state=encoder_states)

    x= temp[0]
    decoder_states=temp[1:]



  dropout = Dropout(rate=do)
  x = dropout(x)
  decoder_dense = Dense(num_decoder_tokens, activation='softmax')
  decoder_outputs = decoder_dense(x)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(
      optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
  )

  if beam_size !=0:
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
        callbacks=[WandbvalCallback(),WandbCallback()]
    )
  else:
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
        callbacks=[WandbCallback()]
    )

  # Save model
  trained_model_artifact = wandb.Artifact(
            MODEL_NAME, type="model",
            description="trained model",
            metadata=dict(cfg))

  model.save(FINAL_MODEL_DIR)
  trained_model_artifact.add_dir(FINAL_MODEL_DIR)
  run.log_artifact(trained_model_artifact)
  run.finish()
  
  model.save("s2s_tamil")

In [ ]:
config_defaults = {
    "epochs" : 10,
    "num_samples" : 10000,
    "embed_dim" : 16,
    "n_encoder" : 2,
    "n_decoder" : 2,
    "drop_out" : 0.2,
    'latent_dim':64,
    'beam_size':0,
    'batch_size':64,
    'cell_type':"LSTM"
    }

train()

In [ ]:
#sweep dictionary
sweep_config={
    'method':'bayes',
    'metric':{
        'name':'val_accuracy',
        'goal':'maximize'},

}

parameters_dict={
    
    'epochs':{
        'values':[10] # ,[36,49,64,64,121],[36,49,49,81,144]
    },
    'num_samples':{
        'values':[10000]
    },
    'embed_dim':{
        'values':[16,32,64]
    },
    'n_encoder':{
        'values':[1,2,3]
    },
    'n_decoder':{
        'values':[1,2,3]
    },
    'drop_out':{
        'values':[0,0.1,0.2]
    },
    'latent_dim':{
      'values':[16,32,64,128] 
    },
    'beam_size':{
        'values':[3]
    },
    'batch_size':{
        'values':[64]
    },
    'cell_type':{
        'values':["LSTM", "GRU"]#, "RNN"]
    }
}

sweep_config['parameters']=parameters_dict

def sweeper(sweep_config,PROJECT_NAME):
  sweep_id=wandb.sweep(sweep_config,project=PROJECT_NAME,entity='cs6910krsrd',)
  wandb.agent(sweep_id,train,project=PROJECT_NAME,entity='cs6910krsrd',)

In [ ]:
sweeper(sweep_config,PROJECT_NAME)

Create sweep with ID: o8vvpo47
Sweep URL: https://wandb.ai/cs6910krsrd/CS6910%20ASSIGNMENT%203/sweeps/o8vvpo47


wandb: Agent Starting Run: c4nznbk5 with config:
wandb: 	batch_size: 64
wandb: 	beam_size: 3
wandb: 	cell_type: LSTM
wandb: 	drop_out: 0.1
wandb: 	embed_dim: 64
wandb: 	epochs: 10
wandb: 	latent_dim: 64
wandb: 	n_decoder: 1
wandb: 	n_encoder: 1
wandb: 	num_samples: 10000
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


{'LSTM': <class 'tensorflow.python.keras.layers.recurrent_v2.LSTM'>, 'GRU': <class 'tensorflow.python.keras.layers.recurrent_v2.GRU'>, 'RNN': <class 'tensorflow.python.keras.layers.recurrent.SimpleRNN'>}
Epoch 1/10
1066/1066 [==============================] - 16s 12ms/step - loss: 1.0782 - accuracy: 0.7259 - val_loss: 0.6522 - val_accuracy: 0.8062
Epoch 2/10
1066/1066 [==============================] - 12s 11ms/step - loss: 0.6709 - accuracy: 0.8032 - val_loss: 0.5374 - val_accuracy: 0.8434
Epoch 3/10
1066/1066 [==============================] - 12s 11ms/step - loss: 0.5551 - accuracy: 0.8371 - val_loss: 0.4413 - val_accuracy: 0.8717
Epoch 4/10
1066/1066 [==============================] - 12s 11ms/step - loss: 0.4586 - accuracy: 0.8645 - val_loss: 0.3675 - val_accuracy: 0.8906
Epoch 5/10
1066/1066 [==============================] - 12s 11ms/step - loss: 0.3894 - accuracy: 0.8840 - val_loss: 0.3139 - val_accuracy: 0.9070
Epoch 6/10
1066/1066 [==============================] - 12s 11ms/s

epoch,9
loss,0.23286
accuracy,0.93181
val_loss,0.20413
val_accuracy,0.94188
_runtime,125
_timestamp,1621467464
_step,9
best_val_loss,0.20413
best_epoch,9


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▄▃▃▂▂▁▁▁
accuracy,▁▃▄▅▆▇▇███
val_loss,█▆▅▄▃▂▂▁▁▁
val_accuracy,▁▃▄▅▆▇▇███
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


Run c4nznbk5 errored: TypeError("'NoneType' object is not subscriptable")
wandb: ERROR Run c4nznbk5 errored: TypeError("'NoneType' object is not subscriptable")
wandb: Agent Starting Run: ywgpqenw with config:
wandb: 	batch_size: 64
wandb: 	beam_size: 3
wandb: 	cell_type: GRU
wandb: 	drop_out: 0.2
wandb: 	embed_dim: 64
wandb: 	epochs: 10
wandb: 	latent_dim: 32
wandb: 	n_decoder: 2
wandb: 	n_encoder: 3
wandb: 	num_samples: 10000
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


{'LSTM': <class 'tensorflow.python.keras.layers.recurrent_v2.LSTM'>, 'GRU': <class 'tensorflow.python.keras.layers.recurrent_v2.GRU'>, 'RNN': <class 'tensorflow.python.keras.layers.recurrent.SimpleRNN'>}
Epoch 1/10
 140/1066 [==>...........................] - ETA: 15s - loss: 2.0183 - accuracy: 0.6360

Inference

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_tamil")

encoder_model = Model(encoder_inputs, encoder_states)

decoder_states_inputs = []
decoder_states = []
d_outputs=embedded_word
for j in range(n_decoder)[::-1]:
  current_state_inputs = [Input(shape=(latent_dim,)) for _ in range(2)]

  temp = output_layers[n_decoder-j-1](d_outputs, initial_state=current_state_inputs)

  d_outputs, cur_states = temp[0], temp[1:]
  decoder_states += cur_states
  decoder_states_inputs += current_state_inputs

decoder_outputs = decoder_dense(d_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())



IndexError: ignored

In [ ]:

def decode_sequence(input_seq, beam_size,encoder_model,decoder_model):
    # Encode the input as state vectors.
    states_value = (encoder_model.predict(input_seq))
    states_value=states_value*n_decoder
    k=beam_size
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index["\t"]

    end_index=target_token_index["\n"] #sample index to check for end

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    sequences=[[list(),0.0,states_value,False]]
    while not stop_condition:
      new_sequences=list()
      stop_condition=True
      #print("a",np.shape([target_seq] + states_value))
      for i in range(len(sequences)):
        seq, score, states_value,stop_seq = sequences[i]
        
        #if particular sequence is finished
        if stop_seq:
          new_sequences.append(sequences[i])
        
        #if particular sequence is unfinished
        else:
          stop_condition=False                                         #if atleast one sequence is unfinished, decoding must continue
          if len(seq)!=0:                                              #dont run for first iteration
            sampled_token_index=seq[-1]
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index

          temp = decoder_model.predict([target_seq] + states_value)
          
          output_tokens=temp[0]
          output_tokens=output_tokens[0, -1, :]
          new_states_value=temp[1:]
          for j in range(len(output_tokens)):
            if j == end_index or len(seq) > max_decoder_seq_length:
              new_seq = [seq + [j], score - np.log(output_tokens[j]),new_states_value,True]
            else:
              new_seq = [seq + [j], score - np.log(output_tokens[j]),new_states_value,False]
            new_sequences.append(new_seq)


      ordered = sorted(new_sequences, key=lambda tup:tup[1])

      # select k best
      sequences = ordered[:k]

    decoded_sentence = ""

    best_seq=sequences[0][0]
    for sampled_token_index in best_seq:
      sampled_char = reverse_target_char_index[sampled_token_index]
      decoded_sentence += sampled_char

    return decoded_sentence

In [ ]:
print(decode_sequence(encoder_input_data[20:21],3))
print(input_texts[20])

போர்ஸ்

force


In [ ]:
class WandbvalCallback(keras.callbacks.Callback):
    def on_train_end(self, epoch, logs=None):
        print("logging")
        wandb.log({"val_Acc_calc": calcValAcc()})

In [ ]:
def calcValAcc():
  encoder_model = Model(encoder_inputs, encoder_states)

  decoder_states_inputs = []
  decoder_states = []
  d_outputs=embedded_word
  for j in range(n_decoder)[::-1]:
    current_state_inputs = [Input(shape=(latent_dim,)) for _ in range(2)]

    temp = output_layers[n_decoder-j-1](d_outputs, initial_state=current_state_inputs)

    d_outputs, cur_states = temp[0], temp[1:]
    decoder_states += cur_states
    decoder_states_inputs += current_state_inputs

  decoder_outputs = decoder_dense(d_outputs)
  decoder_model = Model(
      [decoder_inputs] + decoder_states_inputs,
      [decoder_outputs] + decoder_states)

  # Reverse-lookup token index to decode sequences back to
  # something readable.
  correct_pred=list()

  for seq_index in range(len(val_encoder_input_data)):
    input_seq = val_encoder_input_data[seq_index : seq_index + 1]
    pred=decode_sequence(input_seq, beam_size,encoder_model,decoder_model)
    #print(input_seq,"pred:",pred,"true",val_target_texts[seq_index:seq_index+1])
    #print("yay",pred,val_target_texts[seq_index])
    if pred==val_target_texts[seq_index:seq_index+1]:
      correct_pred.append(1)
      #print("correctpred")
    else:
      correct_pred.append(0)
    
  val_accuracy=np.mean(np.array(correct_pred))

  print(val_accuracy)
  return(val_accuracy)


In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

In [ ]:
epochs=10

Attention

In [ ]:
encoder_inputs=Input(shape=(None,))
x = Embedding(num_encoder_tokens, embed_dim)(encoder_inputs)
encoder_states = []
x, state_h, state_c = CELL[cell_type](latent_dim,dropout=do,return_state=True,return_sequences=True)(x)
encoder= x #[x, state_h, state_c]
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
out_embed=Embedding(num_decoder_tokens, embed_dim)
embedded_word=out_embed(decoder_inputs)
x=embedded_word
output_layers = []
output_layers.append(
      CELL[cell_type](latent_dim,dropout=do, return_sequences=True,return_state=True)
)
x,dh,dc = output_layers[-1](x, initial_state=encoder_states)
decoder= x#[x,dh,dc]

attention = dot([decoder, encoder], axes=[2, 2])
visualization=attention
attention = Activation('softmax')(visualization)
context = dot([attention, encoder], axes=[2,1])
decoder_combined_context = concatenate([context, decoder])

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
output=decoder_dense(decoder_combined_context)

"""#timedistributed?
decoder_dense1=TimeDistributed(Dense(latent_dim, activation="tanh"))
decoder_dense2=TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
output = decoder_dense1(decoder_combined_context)
output = decoder_dense2(output)"""

"""dropout = Dropout(rate=do)
x = dropout(x)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')   #time_distributed?
decoder_outputs = decoder_dense(x)"""

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], output)

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

# Save model
model.save("s2s_tamil")

Epoch 1/20
853/853 [==============================] - 13s 12ms/step - loss: 1.1034 - accuracy: 0.7137 - val_loss: 0.8015 - val_accuracy: 0.7969
Epoch 2/20
853/853 [==============================] - 9s 11ms/step - loss: 0.5164 - accuracy: 0.8472 - val_loss: 0.5569 - val_accuracy: 0.8754
Epoch 3/20
853/853 [==============================] - 9s 11ms/step - loss: 0.2679 - accuracy: 0.9265 - val_loss: 0.4260 - val_accuracy: 0.9087
Epoch 4/20
853/853 [==============================] - 10s 11ms/step - loss: 0.1537 - accuracy: 0.9590 - val_loss: 0.3882 - val_accuracy: 0.9171
Epoch 5/20
853/853 [==============================] - 10s 11ms/step - loss: 0.1138 - accuracy: 0.9692 - val_loss: 0.3984 - val_accuracy: 0.9144
Epoch 6/20
853/853 [==============================] - 9s 11ms/step - loss: 0.0918 - accuracy: 0.9748 - val_loss: 0.4404 - val_accuracy: 0.9031
Epoch 7/20
853/853 [==============================] - 9s 11ms/step - loss: 0.0786 - accuracy: 0.9780 - val_loss: 0.4795 - val_accuracy: 0.8

INFO:tensorflow:Assets written to: s2s_tamil/assets


INFO:tensorflow:Assets written to: s2s_tamil/assets


In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_tamil")

encoder_model = Model(encoder_inputs, [encoder,encoder_states])
#n_decoder=1

decoder_inputs = Input(shape=(None,))
embedded_word=out_embed(decoder_inputs)

decoder_states_inputs = []
decoder_states = []
d_outputs=embedded_word
current_state_inputs = [Input(shape=(latent_dim,)) for _ in range(2)]
encoder_outs=Input(shape=(None,latent_dim,))

temp = output_layers[0](d_outputs, initial_state=current_state_inputs)

d_outputs, cur_states = temp[0], temp[1:]
decoder_states += cur_states
decoder_states_inputs += current_state_inputs

attention_inf = dot([d_outputs, encoder_outs], axes=[2, 2])

attention_inf = Activation('softmax')(attention_inf)

visualization=attention_inf

context_inf = dot([attention_inf, encoder_outs], axes=[2,1])

decoder_combined_context_inf = concatenate([context_inf, d_outputs])

dec_output=decoder_dense(decoder_combined_context_inf)



decoder_model = Model(
    [encoder_outs]+[decoder_inputs] + decoder_states_inputs,
    [dec_output]+ [visualization] + decoder_states)



# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())




In [ ]:
plt.rc('font', family='Vijaya')

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    temp= (encoder_model.predict(input_seq))

    e_out=temp[0]

    states_value=temp[1:]

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index["\t"]

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    vis_list=list()
    end_char_ind=np.max(input_seq[0])
    for i in range(len(input_seq[0])):
      if input_seq[0][i] == end_char_ind:
        break

    while not stop_condition:
        #print("a",np.shape([target_seq] + states_value))
        output_tokens, visualization, h, c = decoder_model.predict([e_out]+[target_seq] + states_value)


        vis_list.append(visualization[0][0][:i+1])


        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    n_dots=np.array(np.shape(vis_list))
    print(n_dots*10)
    mat=np.zeros(n_dots*10)

    print(np.shape(mat))
    xt=list()
    yt=list()

    fig, ax = plt.subplots()
    for i,vis in enumerate(vis_list):
      for j, color in enumerate(vis):
        mat[i*10:i*10+10,j*10:j*10+10]=color
        yt.append(j*10+5)
        xt.append(i*10+5)
        
    
    ax.imshow(mat)
    ax.set_xticks(yt)
    ax.set_yticks(xt)

    print([reverse_input_char_index[token] for token in input_seq[0][:i]])

    ax.set_xticklabels([reverse_input_char_index[token] for token in input_seq[0][:i]])
    ax.set_yticklabels(decoded_sentence)


    
    plt.show()

    return decoded_sentence

In [ ]:
print(decode_sequence(encoder_input_data[20:21]))
print(input_texts[20])

In [ ]:
len(encoder_input_data[25:26][0])

30

Old Inference

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_tamil")

encoder_model = Model(encoder_inputs, encoder_states)

decoder_states_inputs = []
decoder_states = []
d_outputs=embedded_word
for j in range(n_decoder)[::-1]:
  current_state_inputs = [Input(shape=(latent_dim,)) for _ in range(2)]

  temp = output_layers[n_decoder-j-1](d_outputs, initial_state=current_state_inputs)

  d_outputs, cur_states = temp[0], temp[1:]
  decoder_states += cur_states
  decoder_states_inputs += current_state_inputs

decoder_outputs = decoder_dense(d_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())




In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = (encoder_model.predict(input_seq))
    states_value=states_value*n_decoder

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index["\t"]

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        #print("a",np.shape([target_seq] + states_value))
        temp = decoder_model.predict([target_seq] + states_value)
        output_tokens=temp[0]
        print(np.sum(output_tokens))

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = temp[1:]
    return decoded_sentence

In [ ]:
#run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="fit")


# Define an input sequence and process it.
encoder_inputs=Input(shape=(None,))
x = Embedding(num_encoder_tokens, embed_dim)(encoder_inputs)
encoder_states = []
for j in range(n_encoder)[:-1]:
  x, _, _ = CELL[cell_type](latent_dim,dropout=do,return_state=True,return_sequences=True)(x)
x, state_h, state_c = CELL[cell_type](latent_dim,dropout=do,return_state=True,return_sequences=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
out_embed=Embedding(num_decoder_tokens, embed_dim)
embedded_word=out_embed(decoder_inputs)
x=embedded_word
output_layers = []
for j in range(n_decoder):
  output_layers.append(
      CELL[cell_type](latent_dim,dropout=do, return_sequences=True,return_state=True)
  )
  x,dh,dc = output_layers[-1](x, initial_state=encoder_states)

dropout = Dropout(rate=do)
x = dropout(x)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    validation_data=None,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[WandbCallback()]
)

# Save model
model.save("s2s_tamil")